In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

if repo_root not in sys.path:
    sys.path.append(repo_root)  # go to parent dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spf.dataset.spf_dataset import pi_norm

thetas = np.linspace(-np.pi, np.pi, 128 * 4)

c = 3 * 1e8
d = 0.0505  # 505  # seperation
l = 0.12  # lambda
freq = c / l

"""
theta is the angle from array normal to incident
phi is phase difference

delta_distance  = d*sin(theta)
phi = delta_distance * 2pi / lambda = sin(theta)*d*2pi/lambda
theta = arcsin(lambda * phi / (d*2pi))
"""


def theta_to_phi(theta):
    return np.sin(theta) * d * 2 * np.pi / l


def phi_to_theta(phi, limit=False):
    sin_arg = l * phi / (d * 2 * np.pi)
    # assert sin_arg.min()>-1
    # assert sin_arg.max()<1
    if limit:
        edge = 1 - 1e-8
        sin_arg = np.clip(sin_arg, a_min=-edge, a_max=edge)
    v = np.arcsin(l * phi / (d * 2 * np.pi))
    return v, np.pi - v


rnd_noise = np.random.randn(thetas.shape[0])
idx = 0
for noise in np.linspace(0, 1, 200):
    phis_nonoise = theta_to_phi(thetas)
    phis = phis_nonoise + rnd_noise * noise
    phis = pi_norm(phis)
    _thetas = phi_to_theta(phis, limit=True)

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    s = 2
    axs[0].scatter(thetas, phis, s=s, label="phi")
    axs[0].scatter(thetas, pi_norm(_thetas[0]), s=s, label="est. theta0")
    axs[0].scatter(thetas, pi_norm(_thetas[1]), s=s, label="est. theta1")
    axs[0].set_xlabel("theta")
    axs[0].set_ylabel("phi/theta")
    axs[0].plot(thetas, phis_nonoise, label="G.T. Phi")
    axs[0].plot(thetas, thetas, label="G.T. Theta", color="red")
    axs[0].legend(loc=4)
    axs[0].set_title(f"Theta/Phi NoiseInPhi:{noise:0.2f}")

    from spf.data_collector import rx_config_from_receiver_yaml
    from spf.rf import precompute_steering_vectors

    signal_matrix = np.vstack([np.exp(1j * phis), np.ones(phis.shape)])
    nthetas = 64 + 1

    rx_config = rx_config_from_receiver_yaml(
        {
            "receiver-uri": "fake",
            "theta-in-pis": 0,
            "antenna-spacing-m": d,
            "nelements": 2,
            "array-type": "linear",
            "rx-gain-mode": "fast_attack",
            "rx-buffers": 2,
            "rx-gain": -3,
            "buffer-size": 524288,
            "f-intermediate": 100000,
            "f-carrier": freq,
            "f-sampling": 16000000,
            "bandwidth": 300000,
            "motor_channel": 0,
        }
    )

    steering_vectors = precompute_steering_vectors(
        receiver_positions=rx_config.rx_pos,
        carrier_frequency=rx_config.lo,
        spacing=nthetas,
    )
    from spf.rf import beamformer_given_steering_nomean

    beam_sds = beamformer_given_steering_nomean(
        steering_vectors=steering_vectors,
        signal_matrix=signal_matrix,
    )
    axs[1].imshow(beam_sds, aspect="auto", extent=[-np.pi, np.pi, -np.pi, np.pi])
    axs[1].set_title("beamformer")
    fig.savefig(f"tmp_angle_beamformer_noise{idx:02d}.png")
    idx += 1

In [ ]:
np.linspace(-np.pi, np.pi, 5)

In [ ]:
from spf.test import get_phase_diff


plt.plot(thetas, signal_matrix[0].imag)
plt.plot(thetas, np.angle(signal_matrix[0]))
plt.plot(thetas, get_phase_diff(signal_matrix))